In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it


### Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [2]:
attendance_df = pd.read_csv('attendance.csv')
attendance_df.info()
attendance_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4 non-null      object
 1   2018-01-01  4 non-null      object
 2   2018-01-02  4 non-null      object
 3   2018-01-03  4 non-null      object
 4   2018-01-04  4 non-null      object
 5   2018-01-05  4 non-null      object
 6   2018-01-06  4 non-null      object
 7   2018-01-07  4 non-null      object
 8   2018-01-08  4 non-null      object
dtypes: object(9)
memory usage: 416.0+ bytes


,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attendance_df.rename(columns= {'Unnamed: 0': 'student'}, inplace=True)

In [4]:
attendance_T = attendance_df.T
attendance_T.rename(columns= {0: 'Sally', 1: 'Jane', 2: 'Billy', 3: 'John'}, inplace=True)
attendance_T = attendance_T.drop(['student'], axis=0)
attendance_T

attendance_T.info()
attendance_T

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 2018-01-01 to 2018-01-08
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Sally   8 non-null      object
 1   Jane    8 non-null      object
 2   Billy   8 non-null      object
 3   John    8 non-null      object
dtypes: object(4)
memory usage: 320.0+ bytes


,Sally,Jane,Billy,John
2018-01-01,P,A,A,P
2018-01-02,T,P,T,T
2018-01-03,T,T,A,H
2018-01-04,H,T,A,P
2018-01-05,P,T,H,P
2018-01-06,A,T,T,T
2018-01-07,T,A,P,P
2018-01-08,T,T,T,P


In [18]:
att_vals = {'A': -1, 'H': 0.5, 'P': 1, 'T': -0.1}
attendance_vals = attendance_T.replace(to_replace= att_vals, value=None)
attendance_vals

,Sally,Jane,Billy,John
2018-01-01,1.0,-1.0,-1.0,1.0
2018-01-02,-0.1,1.0,-0.1,-0.1
2018-01-03,-0.1,-0.1,-1.0,0.5
2018-01-04,0.5,-0.1,-1.0,1.0
2018-01-05,1.0,-0.1,0.5,1.0
2018-01-06,-1.0,-0.1,-0.1,-0.1
2018-01-07,-0.1,-1.0,1.0,1.0
2018-01-08,-0.1,-0.1,-0.1,1.0


In [17]:
print(f'Billy\'s attendance percentage was: {attendance_vals.Billy.sum()}%')
# The numbers aren't right but at least half the battle is won...

Billy's attendance percentage was: 15.0%


### Coffee Levels

Read the coffee_levels table.

Transform the data so that each carafe is in it's own column.

Is this the best shape for the data?

In [20]:
coffee_df = pd.read_csv('coffee_levels.csv')
coffee_df.info()
coffee_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hour           30 non-null     int64  
 1   coffee_carafe  30 non-null     object 
 2   coffee_amount  30 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 848.0+ bytes


,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [31]:
coffee_piv = pd.pivot_table(data= coffee_df, values= 'coffee_amount', index= 'hour', columns= 'coffee_carafe')
coffee_piv.columns.name = ''
coffee_piv = coffee_piv.reset_index()
coffee_piv.head()


,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947


In [38]:
coffee_repiv = coffee_piv.T
coffee_repiv.reset_index()
coffee_repiv.rename(columns= {0: '8:00', 1: '9:00', 2: '10:00', 3: '11:00', 4: '12:00', 5: '13:00', 6: '14:00', 7: '15:00', 8: '16:00', 9: '17:00'}, inplace=True)
coffee_repiv = coffee_repiv.drop(['hour'], axis=0)
coffee_repiv.reset_index()

,,8:00,9:00,10:00,11:00,12:00,13:00,14:00,15:00,16:00,17:00
0,x,0.816164,0.451018,0.843279,0.335533,0.898291,0.310711,0.507288,0.215043,0.183891,0.391560
1,y,0.189297,0.521502,0.023163,0.235529,0.017009,0.997464,0.058361,0.144644,0.544676,0.594126
2,z,0.999264,0.915990,0.144928,0.311495,0.771947,0.398520,0.864464,0.436364,0.280621,0.436677
